In [1]:
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from pathlib import Path
import pandas as pd, random
import pickle


# ---------- 1. Configuración reproducible ----------
SEED = 0
np.random.seed(SEED)
random.seed(SEED)

# ---------- 2. Cargar MFCC + etiquetas ----------
carpetaD = Path("Data/Audio/mfcc_0.2s/Deceptive")
carpetaT = Path("Data/Audio/mfcc_0.2s/Truthful")

audio, etiquetas = [], []

for archivo in sorted(carpetaD.iterdir()):
    df = pd.read_csv(archivo, sep=" ", header=None, usecols=range(20))
    audio.append(df.values)
    etiquetas.append(0)

for archivo in sorted(carpetaT.iterdir()):
    df = pd.read_csv(archivo, sep=" ", header=None, usecols=range(20))
    audio.append(df.values)
    etiquetas.append(1)

etiquetas = np.array(etiquetas)
n_samples = len(etiquetas)
indices   = np.arange(n_samples)

# ---------- Apartar 15 % TEST ----------
dev_idx, test_idx = train_test_split(
    indices,
    test_size=0.15,          # <-- 15 %
    stratify=etiquetas,
    random_state=SEED
)






In [5]:
print(dev_idx)

[280   8  15 217 249 192 169 203 223  97 167 236  46  10  74   4 227 210
 306 301 308 281 226 181 276 191  23 290  93 180  78  40 273  16 296   1
  66  32 253   0 240 218 300 305  84  95 260 235  96 213  91 272 212  27
 304  54  90 162 295 165 110 138  30 116 262 207 150 291 266  17 205 114
 278 287  37  99 122 244 299 194   6  41  52 104 201 215 211  65 154 134
 157 176 263 182  50 179  59 135 164 121  55 221 128 274 107 149  24 147
 269 131 225 209 168  28 261  19 267 111 184   2  14 255 239  26 155 204
  18 102  73  38  31 113 216  63 152 127  61 189 120 237 148 270 302 199
 220  22 136  82 284 124 105  35  34 293 202 125  76 228  60  75 198 230
 248  29  92 153 100  43 112  11 234 231 232 174 193 132 294 146 185 123
  57 186 279 246 126 159 101 141 140  94 130 129 158 298 171 119 282 258
 170 288 156  44 133  98 109  56 289  85  12 144 195   5 271 143 297 247
 283 307 178   7  51 277  13 250 173 108  79 190 145   3 238  89  83 200
  71 206  45  33 177 175 275  86 183 251  64 106  4

In [2]:
print(len(dev_idx))

262


In [3]:
indices_dev = np.arange(len(dev_idx))
print(indices_dev)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [4]:

# ---------- Folds con el 85 % restante DEV ----------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
folds = [
    (tr, vl)
    for tr, vl in skf.split(indices_dev, etiquetas[dev_idx])
]

In [9]:
print(folds[0][0])

[  1   2   4   5   6   7   9  10  11  12  14  15  17  18  19  21  22  23
  24  25  26  27  29  31  33  34  35  36  37  38  39  40  41  45  46  48
  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  69  70  71  72  73  76  77  79  80  81  82  83  84  85  86  87  89  90
  91  92  94  95  96  97  98  99 101 103 104 105 106 107 108 109 110 111
 112 113 114 118 120 121 122 123 124 127 128 129 130 131 132 133 134 135
 136 137 138 141 143 144 145 147 148 149 150 151 154 155 156 157 158 159
 160 161 162 163 164 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 181 183 184 186 188 189 190 191 192 193 194 196 197 198 199 200 201
 204 205 206 207 208 209 210 211 213 214 215 216 218 219 220 221 222 223
 224 226 227 228 230 233 234 235 236 237 238 240 241 242 243 244 245 246
 247 248 249 253 254 256 257 258 259 260 261]


In [10]:
for fold in folds:
    print(len(fold[0]), len(fold[1]))

209 53
209 53
210 52
210 52
210 52


In [11]:
with open('fold_indices.pkl', 'wb') as f:
    pickle.dump(folds, f)


In [12]:
np.savez("holdout_indices.npz", dev=dev_idx, test=test_idx)


In [13]:

print(
    f"Total muestras: {n_samples}\n"
    f"DEV: {len(dev_idx)}  |  TEST: {len(test_idx)}\n"
    "fold_indices.npz y test_holdout.npz guardados."
)

Total muestras: 309
DEV: 262  |  TEST: 47
fold_indices.npz y test_holdout.npz guardados.
